In [1]:
import os, sys

In [2]:
model_id = 'ContinuousSBM_ContinuousVPSDE_I_BPROJ_bottleneck_32_firstc_10_phi_beta_cosine_betamax_0.5_betamin_0.001'
output_dir ='/mnt/home/dheurtel/ceph/04_inference/sbc/'+model_id
num_samples_per_chain = 500
num_sims_per_gpu = 256
nb_gpus = 4

In [3]:
script_template = \
"""#!/bin/bash
#SBATCH -J sbc_{{noise_level}}_{{count}}
#SBATCH --gpus=1
#SBATCH --cpus-per-gpu=16
#SBATCH --time=48:00:00
#SBATCH --partition=gpu
#SBATCH --constraint='a100-80gb|h100'
#SBATCH -o jobs/sbc_MN_{{noise_level}}_{{count}}.log

source ~/.bashrc
source /mnt/home/dheurtel/venv/genv_DL/bin/activate
python sbc_with_MN.py --model_id {{model_id}} --num_chain={{num_sims_per_gpu}} --num_sample={{num_samples_per_chain}} --noise_level={{noise_level}} --save_path={{save_path}}
"""

In [4]:
keys_substitutes = {"{{model_id}}": model_id,
                    "{{num_sims_per_gpu}}": num_sims_per_gpu,
                    "{{num_samples_per_chain}}": num_samples_per_chain,
                    "{{save_path}}": os.path.join(output_dir, "tmp.pt"),
                    "{{noise_level}}": 0.1,
                    "{{count}}": 0}

In [5]:
noise_levels = [0.1, 0.2, 0.3, 0.5]

for noise_level in noise_levels:
    for gpu_id in range(nb_gpus):
        keys_substitutes["{{noise_level}}"] = noise_level
        keys_substitutes["{{count}}"] = gpu_id
        keys_substitutes["{{save_path}}"] = os.path.join(output_dir, f"sbc_MN_sims_{noise_level}_{gpu_id}.pt")
        script = script_template
        for key, value in keys_substitutes.items():
            script = script.replace(key, str(value))
        with open(f'./jobs/sbc_MN_{noise_level}_{gpu_id}.sh', 'w') as f:
            f.write(script)
        os.system(f'sbatch ./jobs/sbc_MN_{noise_level}_{gpu_id}.sh')

Submitted batch job 3114296
Submitted batch job 3114297
Submitted batch job 3114298
Submitted batch job 3114299
Submitted batch job 3114300
Submitted batch job 3114301


Submitted batch job 3114302
Submitted batch job 3114303
Submitted batch job 3114304
Submitted batch job 3114305
Submitted batch job 3114306
Submitted batch job 3114307
Submitted batch job 3114308
Submitted batch job 3114309
Submitted batch job 3114310
Submitted batch job 3114311
